In [4]:
import sys
sys.path.insert(0, 'Import_data')
from train_test_data import train_test_data

A_train, A_test, label_train, label_test, v_train, v_test = train_test_data()

In [9]:
import numpy as np
import time
from itertools import combinations
from numpy.linalg import inv

N0 = 28 * 28 + 1  # dimension layer 0
M = 0;  # Se actualiza posteriormente


def sigmoid(x):
    return 1.0/(1 + np.exp(-x/30))    


def sigmoid_d(y):
    return y*(1-y)*1/30


def square(x):
    return x*x


sigmoid_v = np.vectorize(sigmoid)
sigmoid_d_v = np.vectorize(sigmoid_d)
square_v = np.vectorize(square)


def rel_error(new_error, old_error):
    return abs(new_error - old_error) / new_error


def obtain_y(X, weights):
    return sigmoid_v(X.dot(weights))


def obtain_Ek(X, Y, separador, d0, d1):  # Ek Matriz, asumiendo label vector fila de dimension M
    M = len(X)
    Ek = np.zeros(M)
    for k in range(M):
        zk = 0
        if (k >= separador):
            zk = 1
        Ek[k] = (Y[k] - zk)
    return Ek


def calculate_error(Ek, X):
    M = len(X)
    aux = square_v(Ek)
    return 0.5 * float(np.ones(M).dot(aux))


def hessian(X, Y, Ek):
    M = len(X)
    Y_d = sigmoid_d_v(Y)
    H = np.zeros((785, 785))
    for i in range(785):
        for j in range(785):
            if (i <= j):
                for k in range(M):
                    H[i, j] += Y_d[k] * X[k][i] * X[k][j] * (Y_d[k] + Ek[k] * (1 - 2 * Y[k]))
                H[j, i] = H[i, j]

def grad(X, Y, Ek):
    M = len(X)
    grad_weights = np.zeros(N0)
    Y_d = sigmoid_d_v(Y)
    for i in range(N0):
        aux = 0
        for k in range(M):
            aux += Ek[k]*X[k][i]*Y_d[k]
        grad_weights[i] = aux
    return grad_weights


# Entrena el clasificador binario de labels d0, d1 y devuelve los pesos
# Supongo que el output correcto es d1 --> output = 1
def train_classifier(X, separador, d0, d1):
    M = len(X)
    weights = np.random.rand(N0)  # [i, t]
    Y = obtain_y(X, weights)
    Ek = obtain_Ek(X, Y, separador, d0, d1)
    eps = 1e-5
    n_iteraciones = 100
    cont = 0
    learning_rate = 0.01
    old_error = np.inf
    new_error = calculate_error(Ek, X)

    while (cont < n_iteraciones): #and rel_error(new_error, old_error) > eps):
        print("inside")
        cont += 1
        start = time.time()
        gradiente = grad(X, Y, Ek)
        #Hessian = hessian(X, Y, Ek)
        #weights -= learning_rate * inv(Hessian).dot(gradiente)
        weights -= learning_rate * gradiente
        Y = obtain_y(X, weights)
        Ek = obtain_Ek(X, Y, separador, d0, d1)
        old_error = new_error
        new_error = calculate_error(Ek, X)
        end = time.time()
        print(old_error, new_error)
        print('Training ' + str(d0) + ' vs ' + str(d1) + ': -->', 'rel Error = ' + str(rel_error(new_error, old_error)) + ',', 'elapsed time = '+str(end-start)+',', cont, '\n')
        print(Y)

    return weights


def test_data(binary_nets, foto):
    v = 10 * [0]
    for digits in binary_nets:
        result = float(sigmoid(weights.dot(foto)))
        if result >= .5:
            v[digits[1]] += 1
        else:
            v[digits[0]] += 1
    return np.argmax(v)
    
    
    
def train_binary_nets():
    iterator = combinations('0123456789', 2)  # iterador con (0,1), (0,2), ... , (8,9)
    binary_nets = {}  # diccionario con todos los classificadores binarios
    for digits in iterator:
        digits = tuple(map(int,digits)) #pasar a integer
        d0 = digits[0]
        d1 = digits[1]
        X1 = A_train[d0]
        X2 = A_train[d1]
        X = np.concatenate((X1, X2), axis = 0)
        separador = len(X1)
        binary_nets[digits] = train_classifier(X, separador, d0, d1)
    
    return binary_nets
    
def main ():
    binary_nets = train_binary_nets()   
    print("End training")
     
    N_aciertos = 0
    N_test = len(label_test)
    precision = N_aciertos/N_test
    for i in range(10):
        print(i)
        for foto in A_test[i]:
            prediction = test_data(binary_nets, foto)
            if (prediction == i): N_aciertos += 1
        print("Precision en digito ", i, " : ", precision) 
    print("Precision Final: ", precision)

In [ ]:
main()

inside
1011.7842246418073 998.202656703313
Training 0 vs 1: --> rel Error = 0.013606022632066662, elapsed time = 5.705934047698975, 1 

[0.62501705 0.606564   0.64629232 ... 0.57588208 0.57715933 0.57356221]
inside
998.202656703313 984.5800141113676
Training 0 vs 1: --> rel Error = 0.013835993415162364, elapsed time = 6.248281002044678, 2 

[0.61847363 0.60189247 0.64127934 ... 0.57571131 0.57723202 0.57386339]
inside
984.5800141113676 970.9338063821364
Training 0 vs 1: --> rel Error = 0.014054725089941345, elapsed time = 6.432437896728516, 3 

[0.61187795 0.59719631 0.63622619 ... 0.57553716 0.57730085 0.57415969]
inside
970.9338063821364 957.2820654932417
Training 0 vs 1: --> rel Error = 0.014260938735816152, elapsed time = 6.294646978378296, 4 

[0.60523794 0.59248093 0.63113892 ... 0.57536184 0.57736813 0.57445357]
inside
957.2820654932417 943.6431908714776
Training 0 vs 1: --> rel Error = 0.014453423448293277, elapsed time = 6.293414115905762, 5 

[0.59856183 0.58775185 0.62602382

570.7259447665135 563.796584153226
Training 0 vs 1: --> rel Error = 0.012290533160456779, elapsed time = 6.400478839874268, 41 

[0.39287887 0.44573731 0.4689549  ... 0.58254868 0.59361422 0.5993068 ]
inside
563.796584153226 557.0339855272193
Training 0 vs 1: --> rel Error = 0.012140369890727707, elapsed time = 5.8067967891693115, 42 

[0.38891351 0.44304472 0.46597171 ... 0.58324249 0.59455109 0.60048433]
inside
557.0339855272193 550.4326765972069
Training 0 vs 1: --> rel Error = 0.011992945205982276, elapsed time = 6.332536697387695, 43 

[0.38504075 0.44041646 0.46306236 ... 0.58395755 0.59550845 0.60168203]
inside
550.4326765972069 543.9872961302099
Training 0 vs 1: --> rel Error = 0.011848402550662166, elapsed time = 5.855874061584473, 44 

[0.38125846 0.43785102 0.46022517 ... 0.58469303 0.59648539 0.60289894]
inside
543.9872961302099 537.6926089902223
Training 0 vs 1: --> rel Error = 0.011706850781915853, elapsed time = 5.815553188323975, 45 

[0.37756449 0.43534683 0.45745843 .

382.4564218546002 379.2924312222205
Training 0 vs 1: --> rel Error = 0.008341823806460218, elapsed time = 6.388485908508301, 81 

[0.28683687 0.37423167 0.39133086 ... 0.61965351 0.63939301 0.65386446]
inside
379.2924312222205 376.17735918636276
Training 0 vs 1: --> rel Error = 0.008280859971464927, elapsed time = 5.893619060516357, 82 

[0.28512257 0.37307888 0.39012056 ... 0.62067225 0.64059346 0.65525102]
inside
376.17735918636276 373.1100512623054
Training 0 vs 1: --> rel Error = 0.008220920111050552, elapsed time = 5.835773944854736, 83 

[0.28343716 0.3719451  0.38893193 ... 0.62168998 0.64179104 0.65663279]
inside
373.1100512623054 370.08939398118207
Training 0 vs 1: --> rel Error = 0.008161966622790864, elapsed time = 5.786783933639526, 84 

[0.28177984 0.37082975 0.3877643  ... 0.62270646 0.64298551 0.65800954]
inside
370.08939398118207 367.1143128953684
Training 0 vs 1: --> rel Error = 0.008103963755457246, elapsed time = 5.64815092086792, 85 

[0.28014984 0.36973226 0.386617

730.8996643330036 723.6318331661355
Training 0 vs 2: --> rel Error = 0.010043548160490438, elapsed time = 5.592437982559204, 21 

[0.52499818 0.54111329 0.56447968 ... 0.58348014 0.63442237 0.63521096]
inside
723.6318331661355 716.5207596549038
Training 0 vs 2: --> rel Error = 0.00992444868541779, elapsed time = 5.512129068374634, 22 

[0.51958068 0.5380672  0.56031392 ... 0.58203879 0.63315371 0.63243617]
inside
716.5207596549038 709.5658625882542
Training 0 vs 2: --> rel Error = 0.009801622982932934, elapsed time = 5.70264196395874, 23 

[0.51424288 0.53509055 0.55621987 ... 0.58066523 0.63196159 0.62975214]
inside
709.5658625882542 702.7660284852341
Training 0 vs 2: --> rel Error = 0.009675815032887523, elapsed time = 5.402555227279663, 24 

[0.50898695 0.53218392 0.55219903 ... 0.57935952 0.63084613 0.62715993]
inside
702.7660284852341 696.1196623286363
Training 0 vs 2: --> rel Error = 0.009547735132727844, elapsed time = 5.376451015472412, 25 

[0.50381468 0.52934769 0.54825264 ..

535.194608591154 531.9548867697408
Training 0 vs 2: --> rel Error = 0.006090219118177696, elapsed time = 6.087430953979492, 61 

[0.37102887 0.46744202 0.45213801 ... 0.57017266 0.6334903  0.58814202]
inside
531.9548867697408 528.762217985341
Training 0 vs 2: --> rel Error = 0.006038004751104176, elapsed time = 6.436852931976318, 62 

[0.3685296  0.46654667 0.45046546 ... 0.57065974 0.63436938 0.58821595]
inside
528.762217985341 525.6151769917473
Training 0 vs 2: --> rel Error = 0.005987348028276405, elapsed time = 6.183632135391235, 63 

[0.36607587 0.46568007 0.44882984 ... 0.57116919 0.63527162 0.58832647]
inside
525.6151769917473 522.5124087913453
Training 0 vs 2: --> rel Error = 0.005938171320331339, elapsed time = 5.658084154129028, 64 

[0.36366639 0.46484117 0.44723001 ... 0.5717     0.63619582 0.58847213]
inside
522.5124087913453 519.4526250586329
Training 0 vs 2: --> rel Error = 0.005890399980877984, elapsed time = 5.970742702484131, 65 

[0.3612999  0.46402899 0.44566484 ...

926.3948881528082 917.1269655789881
Training 0 vs 3: --> rel Error = 0.01010538662765106, elapsed time = 6.270053863525391, 1 

[0.64723194 0.6240826  0.66734875 ... 0.63420343 0.66134511 0.73500685]
inside
917.1269655789881 907.8626567121432
Training 0 vs 3: --> rel Error = 0.010204526861360185, elapsed time = 5.7667059898376465, 2 

[0.64144726 0.6201017  0.66231198 ... 0.63232275 0.65892749 0.73040589]
inside
907.8626567121432 898.6129620520518
Training 0 vs 3: --> rel Error = 0.010293302067409601, elapsed time = 6.380409002304077, 3 

[0.635635   0.61612607 0.65725169 ... 0.63046384 0.65653591 0.72578298]
inside
898.6129620520518 889.3888853059468
Training 0 vs 3: --> rel Error = 0.010371252551612356, elapsed time = 8.275233030319214, 4 

[0.62980104 0.61215963 0.65217279 ... 0.62862915 0.65417375 0.72114353]
inside
889.3888853059468 880.2013511491735
Training 0 vs 3: --> rel Error = 0.010437991426368755, elapsed time = 6.231053113937378, 5 

[0.62395137 0.60820635 0.6470803  ... 0